In [9]:
import pandas as pd
import uuid
#from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow_decision_forests as tfdf

# Used in FL
import nest_asyncio
nest_asyncio.apply()
import collections
import tensorflow_federated as tff
import tensorflow as tf

np.random.seed(0)
import random

In [10]:
# Testing TFF
print(tff.federated_computation(lambda: 'Hello, World!')())

2023-04-29 12:11:19.523411: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:11:19.525331: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
E0429 12:11:19.764594400     338 socket_utils_common_posix.cc:221] check for SO_REUSEPORT: {"created":"@1682784679.764564500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":199,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
E0429 12:11:19.765454700     338 socket_utils_common_posix.cc:327] setsockopt(TCP_USER_TIMEOUT) Protocol not available


b'Hello, World!'


In [11]:
# Read the CSV files into separate dataframes
df_ey = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_younger.csv')
df_eo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/EIRLI_older.csv')
df_ly = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_younger.csv')
df_lo = pd.read_csv('/home/draxtik20/Federated_Learning_CDI/Datasets/LASER_older.csv')

In [12]:
# Concatenate the dataframes into one dataframe with matching headers
df = pd.concat([df_ey, df_eo, df_ly, df_lo], ignore_index=True, sort=False)

In [13]:
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
0,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
1,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
2,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
3,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
4,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
870,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
871,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0
872,1,1,3,2,1,1,48.0,0.565451,13.000000,0.310525,191.393162,3.060358,1.0,NoDx,55.0


In [14]:
# Generate a unique client ID for each row
df['client_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

# Set the index of the dataframe to the client_id column
df.set_index('client_id', inplace=True)

In [15]:
# Print the dataframe
df

,Level.gender,Level.ethnic,Level.med,Level.fed,Level.income,FamHist,Percentile,GCC,Degree,HC,Between,PathLength,CombWords,AnyLangorReadDxOnly,GramComplex
client_id,,,,,,,,,,,,,,,
ae9ceaee-6ce0-4f42-b0f1-616758270aaf,0,4,4,4,3,0,67.0,0.431818,3.632653,0.258744,27.061224,2.870240,2.0,NoDx,NaN
f30b81a8-8a15-4609-b5fa-59464e924a45,1,4,6,6,3,0,95.0,0.597950,12.480000,0.336051,144.480000,2.987580,1.0,NoDx,NaN
afad206d-45c1-4a80-8fe6-23ac92df220b,1,4,5,4,2,1,10.0,0.000000,1.000000,0.416667,0.250000,1.333333,1.0,NoDx,NaN
c918afd9-7635-42ef-9e4e-da0f56125430,0,4,3,3,2,0,37.0,0.473684,2.000000,0.212454,1.142857,1.615385,1.0,NoDx,NaN
dbd15d32-a137-4e55-bf03-a95a1f73d856,1,4,6,3,3,0,92.0,0.543568,8.125000,0.293187,128.159722,3.270546,2.0,NoDx,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d7db61ed-8dd0-4898-a91d-0883cc76f57e,0,4,6,4,3,1,1.0,0.691011,2.666667,0.140688,5.416667,2.266234,1.0,NoDx,15.0
33fc49e2-427a-4720-bf2c-0129de943698,0,4,4,6,3,0,95.0,0.516904,17.176471,0.319899,300.974790,3.040681,1.0,NoDx,55.0
f5163736-a1e1-43c5-aee4-daa22533fa9d,1,4,5,3,2,0,1.0,0.818182,1.933333,0.106475,5.166667,2.823529,1.0,Dx,39.0


In [16]:
# Get the columns with NaN values
nan_cols = df.columns[df.isnull().any()]

# Print the columns with NaN values as a list
print(nan_cols.tolist())

['GramComplex']


In [17]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
# Fill missing values in the numeric columns with the mean value
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [18]:
# Replace the 'NoDx' value with 0 and the 'Dx' value with 1
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].replace("NoDx", 0).replace("Dx", 1)
df["AnyLangorReadDxOnly"] = df["AnyLangorReadDxOnly"].astype(int)

In [19]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
print(len(train_data), len(test_data), type(train_data))

699 175 <class 'pandas.core.frame.DataFrame'>


In [21]:
print(test_data.dtypes)

Level.gender             int64
Level.ethnic             int64
Level.med                int64
Level.fed                int64
Level.income             int64
FamHist                  int64
Percentile             float64
GCC                    float64
Degree                 float64
HC                     float64
Between                float64
PathLength             float64
CombWords              float64
AnyLangorReadDxOnly      int64
GramComplex            float64
dtype: object


In [22]:
# ----------------------------------------------------Creating Federated Data--------------------------------------------------

#Get a list of client_names
client_names = df.index.values

# Get a numpy array of each client's values for all collumns, except the label column
client_data = df[["Level.gender", "Level.ethnic", "Level.med", "Level.fed", "Level.income", "FamHist",\
                 "Percentile", "GCC", "Degree", "HC", "Between", "PathLength", "CombWords", "GramComplex"]].values

# Get a numpy array of each client's value for only the label column
client_labels = df["AnyLangorReadDxOnly"].values

# Get an array of a tuple(client_data element, respective client_label)
data = list(zip(client_data, client_labels))

#shard data and place at each client
shards = [data[i:i + 1] for i in range(0, len(data), 1)]

# Assert number of clients must equal number of shards
assert(len(shards) == len(client_names))

# Create a dictionary with the client names as keys and the data columns as the values
clients = {client_names[i] : shards[i] for i in range(len(client_names))}
clients = {k: v[0] for k, v in clients.items()}


# Batch the data_shard into a an array of tensorflow dataset
def batch_data(data_shard, bs=32):
    
    # Create a dataset from the labels and features of this client
    dataset = tf.data.Dataset.from_tensor_slices(collections.OrderedDict(
        x=tf.constant([data_shard[0]], dtype=tf.float64),
        y=tf.constant([[data_shard[1]]], dtype=tf.int64)
    ))
    
    # Shuffle and batch the dataset.
#     return dataset.shuffle(32, seed=1).batch(bs)
    return dataset.batch(bs)

# Run the function for each row in the clients dictionary, and add it back to a dictionary clients_batched
federated_data = []
for (client_name, data) in clients.items():
    federated_data.append(batch_data(data))


print(federated_data)

[<_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])>, <_BatchDataset element_spec=OrderedDict([('x', TensorSpec(shape=(None, 14), dtype=tf.float64, name=None)), ('y', TensorSpec(shape=(Non

In [16]:
# -----------------------------------------------Create a Federated Model-------------------------------------------------------

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# This function creates a Sequential keras model
def create_compiled_keras_model():

    model = Sequential([
        Dense(64, activation='relu', input_shape=(14,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    return model

# Summarize the model (should be the same as the centralized model)
m = create_compiled_keras_model()
m.summary() 


# Shuffle the data randomly
random.seed(42)
random.shuffle(federated_data)

# Calculate the number of elements to select when splitting train and test data into a 80/20 split
n = int(len(federated_data) * 0.2)

# Use random.sample to randomly select n elements from the list
federated_test_data = federated_data[:n] # Represents central data that was collected with consent (without FL)
federated_train_data = federated_data[n:] # Represents FL client data

# This function creates a Sequential keras model and wraps it in a TFF model.
def model_fn():
  keras_model = create_compiled_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec, # Set the dataset's specifications
      loss=tf.keras.losses.BinaryCrossentropy(), # Set the loss function
      metrics=[tf.keras.metrics.BinaryAccuracy()]) # Set the metric function to measure accuracy

# Build the training process with the TFF model (as a function), the client-side optimizer and server-side optimizer
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate= 0.001),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate= 0.001))

# Take 10 samples of the training process and store the results in a file samples.txt
with open('samples.txt', 'w') as f:
    NUM_SAMPLES = 10
    for sample_num in range(NUM_SAMPLES):
        # For our implementation, it would be a challenge to pick randomly selected datasets, to simulate real world interactions.
        train_state = training_process.initialize()
        
        # Number of Federated learning computations to run
        NUM_ROUNDS = 10
        
        # Run 10 FL computations with the client data.
        for round_num in range(NUM_ROUNDS):
          result = training_process.next(train_state, federated_train_data)
          train_state = result.state
          train_metrics = result.metrics
          print('round {:2d}, metrics={}'.format(round_num, train_metrics), file=f)

        print("-----------------------------------End----------------------------------", file=f)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                960       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,073
Trainable params: 3,073
Non-trainable params: 0
_________________________________________________________________


2023-04-29 12:11:29.335330: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:11:29.335804: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:11:29.392218: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:11:29.392595: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:11:29.902235: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:11:29.902607: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:11:30.027039: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:11:30.027307: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


2023-04-29 12:11:36.383384: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.475868: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.655609: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.735181: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.806407: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.814078: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:36.831221: W external/org_tensorflow/tens

2023-04-29 12:11:39.123343: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.149538: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.151232: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.160023: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.419273: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.433449: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:39.436181: W external/org_tensorflow/tens

2023-04-29 12:11:42.543849: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.583997: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.617912: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.795507: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.864482: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.866129: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:11:42.881818: W external/org_tensorflow/tens

...

2023-04-29 12:17:02.551870: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.581670: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.766122: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.770133: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.782390: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.798803: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:02.837257: W external/org_tensorflow/tens

2023-04-29 12:17:05.624342: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.656684: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.658313: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.669112: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.670542: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.693940: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:05.701026: W external/org_tensorflow/tens

2023-04-29 12:17:08.597971: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.603106: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.607343: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.614106: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.620817: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.908095: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:08.909203: W external/org_tensorflow/tens

2023-04-29 12:17:10.521845: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.545781: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.552794: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.615428: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.636616: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.639956: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:10.651854: W external/org_tensorflow/tens

2023-04-29 12:17:13.621799: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.668510: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.696119: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.702420: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.835949: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.929347: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:13.953688: W external/org_tensorflow/tens

In [24]:
# Build an evaluation process
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

# Initialize the evaluation process
evaluation_state = evaluation_process.initialize()

# Get the model weights from the final state of the training process
model_weights = training_process.get_model_weights(train_state)

# Set the model weights for the evaluation process
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

# Run an evaluation process, using the federated test data.
# This process happens on the server-side.
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

print(str(evaluation_output.metrics))

2023-04-29 12:17:16.554693: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:17:16.555106: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:17:16.570062: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:17:16.570385: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:17:16.599752: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:17:16.600102: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-04-29 12:17:16.624360: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2023-04-29 12:17:16.624578: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)])), ('total_rounds_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


2023-04-29 12:17:17.643249: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:17.660463: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled


In [25]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)
print(str(evaluation_output.metrics))

2023-04-29 12:17:17.977800: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled
2023-04-29 12:17:17.987444: W external/org_tensorflow/tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled


OrderedDict([('distributor', ()), ('client_work', OrderedDict([('eval', OrderedDict([('current_round_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)])), ('total_rounds_metrics', OrderedDict([('binary_accuracy', 0.9310345), ('loss', 0.99576825), ('num_examples', 174), ('num_batches', 174)]))]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', ())])


In [28]:
import re
import csv

# Open the file and read the contents
with open('samples.txt', 'r') as file:
    content = file.read()

# Use regular expressions to extract the data
pattern = r"round\s+(\d+),\s+\('binary_accuracy',\s+([\d.]+)\),\s+\('loss',\s+([\d.]+)\)"
data = re.findall(pattern, content)
print(data)
# Create a CSV file and write the data
with open('results2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['round', 'binary_accuracy', 'loss'])
    for row in data:
        writer.writerow(row)

[('0', '0.5271429', '1.3698748'), ('1', '0.75', '0.7620509'), ('2', '0.91571426', '0.5517806'), ('3', '0.94', '0.52911127'), ('4', '0.9442857', '0.56513923'), ('5', '0.9442857', '0.61564714'), ('6', '0.9442857', '0.6685063'), ('7', '0.9442857', '0.71415985'), ('8', '0.9442857', '0.745292'), ('9', '0.9442857', '0.76249915'), ('0', '0.5271429', '1.3698748'), ('1', '0.75', '0.7620509'), ('2', '0.91571426', '0.5517806'), ('3', '0.94', '0.52911127'), ('4', '0.9442857', '0.56513923'), ('5', '0.9442857', '0.61564714'), ('6', '0.9442857', '0.6685063'), ('7', '0.9442857', '0.71415985'), ('8', '0.9442857', '0.745292'), ('9', '0.9442857', '0.76249915'), ('0', '0.5271429', '1.3698748'), ('1', '0.75', '0.7620509'), ('2', '0.91571426', '0.5517806'), ('3', '0.94', '0.52911127'), ('4', '0.9442857', '0.56513923'), ('5', '0.9442857', '0.61564714'), ('6', '0.9442857', '0.6685063'), ('7', '0.9442857', '0.71415985'), ('8', '0.9442857', '0.745292'), ('9', '0.9442857', '0.76249915'), ('0', '0.5271429', '1.36

In [5]:
# Reference: 

# Why use Sequential models for TFF, instead of normal Random Forest: https://github.com/tensorflow/federated/issues/3847
# TFF model Evaluation Process: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#evaluation
